In [1]:
# importanto bibliotecas
import pandas as pd
import numpy as np
import psycopg2

#pacotes gráficos
import seaborn as sns
import matplotlib.pyplot as plt

#importando bibliotecas estatísticas
import scipy.stats as stats

In [2]:
# conexão redshift 
connection = psycopg2.connect(dbname="dw_amorsaude",

                  user="lucasmateus",

                  password="df93aS@aK0sd",

                  host ="amorsaude-cluster-redshift.cmc4ev2oyczd.us-east-1.redshift.amazonaws.com",

                  port = 5439)

In [3]:
# definindo query
query = """
select 
	cp.datapagamento,
	cp.id,
	cp.nome_fantasia nome_unidade,
	case when cp.subcategoria in ('Royalties','Royalties (Split)') then 'Royalties' 
	     else cp.categoria end categoria,
	cp.subcategoria,
	cp.valorpago
from tb_consolidacao_contas_a_pagar_hist cp
where cp.situacaoconta <> 'Em aberto'
and cp.datapagamento <= current_date
order by cp.datapagamento"""

In [4]:
# setando o df
df = pd.read_sql_query(query,con=connection)
df

/tmp/ipykernel_12145/3854971275.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,con=connection)


,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75
...,...,...,...,...,...,...
5175172,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00
5175173,2023-08-23,19843.0,AmorSaúde Peruíbe,Repasses,Repasse de Consultas,25.00
5175174,2023-08-23,19969.0,AmorSaúde Porto Alegre Centro,Repasses,None,1440.50
5175175,2023-08-23,19969.0,AmorSaúde Porto Alegre Centro,Repasses,None,528.00


In [5]:
df.isna().mean()*100

datapagamento     0.000000
id                0.063978
nome_unidade      0.063978
categoria         2.575062
subcategoria     51.822653
valorpago         0.000000
dtype: float64

In [6]:
df_null = df[df['categoria'].isna() | df['subcategoria'].isna()]
df_null

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago
8,2021-01-01,19358.0,AmorSaúde São Leopoldo,Repasses,None,58000.00
10,2021-01-01,19626.0,AmorSaúde Porto Velho,None,None,200.00
17,2021-01-01,19626.0,AmorSaúde Porto Velho,None,None,150.00
27,2021-01-02,19289.0,AmorSaúde Franco da Rocha,Repasses,None,1530.00
28,2021-01-02,19563.0,AmorSaúde RJ Campo Grande,Repasse de Exames,None,165.00
...,...,...,...,...,...,...
5175161,2023-08-23,19969.0,AmorSaúde Porto Alegre Centro,None,None,114.35
5175162,2023-08-23,19421.0,AmorSaúde SP Capela do Socorro,None,None,162.50
5175174,2023-08-23,19969.0,AmorSaúde Porto Alegre Centro,Repasses,None,1440.50
5175175,2023-08-23,19969.0,AmorSaúde Porto Alegre Centro,Repasses,None,528.00


In [7]:
df = df.dropna()
df['qtd_lancamentos'] = 1

/tmp/ipykernel_12145/813328229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['qtd_lancamentos'] = 1


In [8]:
# Calculate quartiles using the groupby and agg functions for each quartile
q1_df = df.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['valorpago'].quantile(0.25).reset_index()
median_df = df.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['valorpago'].median().reset_index()
q3_df = df.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['valorpago'].quantile(0.75).reset_index()

# Rename columns to reflect the quartiles
q1_df.rename(columns={'valorpago': 'Q1'}, inplace=True)
median_df.rename(columns={'valorpago': 'mediana'}, inplace=True)
q3_df.rename(columns={'valorpago': 'Q3'}, inplace=True)

# Merge the quartile DataFrames to create the final result
quartiles_df = q1_df.merge(median_df, on=['id', 'nome_unidade', 'categoria', 'subcategoria']).merge(q3_df, on=['id', 'nome_unidade', 'categoria', 'subcategoria'])

#criando a coluna de IQR
quartiles_df['IQR'] = (quartiles_df['Q3'] - quartiles_df['Q1'])

# Calculate lower and upper fences using IQR
iqr_multiplier = 1.5  # Adjust as needed
quartiles_df['limite_inferior'] = quartiles_df['Q1'] - iqr_multiplier * quartiles_df['IQR']
quartiles_df['limite_superior'] = quartiles_df['Q3'] + iqr_multiplier * quartiles_df['IQR']

quartiles_df

,id,nome_unidade,categoria,subcategoria,Q1,mediana,Q3,IQR,limite_inferior,limite_superior
0,0.0,CENTRAL AMORSAÚDE,Funcionários,Vale Transporte,1315.2000,1315.20,1315.2000,0.0000,1315.20000,1315.20000
1,18263.0,AmorSaúde Carapicuíba,Administrativas,Advogados,2044.4500,2044.45,2044.4500,0.0000,2044.45000,2044.45000
2,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel Imóvel,600.0000,857.58,7995.6000,7395.6000,-10493.40000,19089.00000
3,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel de Equipamentos,600.0000,900.59,2790.0000,2190.0000,-2685.00000,6075.00000
4,18263.0,AmorSaúde Carapicuíba,Administrativas,Alvarás,240.6400,698.00,803.1275,562.4875,-603.09125,1646.85875
...,...,...,...,...,...,...,...,...,...,...
33936,19990.0,AmorSaúde Piracicaba Vila Sônia,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,143.7500,234.50,325.2500,181.5000,-128.50000,597.50000
33937,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Consultas,420.0000,500.00,600.0000,180.0000,150.00000,870.00000
33938,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Procedimentos,60.0000,155.00,261.5000,201.5000,-242.25000,563.75000
33939,19990.0,AmorSaúde Piracicaba Vila Sônia,Sócios,Transação Entre Contas Médica e Odonto,283.7175,1909.98,3590.7150,3306.9975,-4676.77875,8551.21125


In [9]:
# Merge df_iqr with quartiles_df based on specified columns
df_iqr = df.merge(quartiles_df, on=['id', 'nome_unidade', 'categoria', 'subcategoria'], how='left')
df_iqr

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,Q1,mediana,Q3,IQR,limite_inferior,limite_superior
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,49.25,150.00,210.00,160.75,-191.875,451.125
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,746.32,755.27,765.26,18.94,717.910,793.670
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,150.00,150.00,200.00,50.00,75.000,275.000
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,35.50,92.50,160.00,124.50,-151.250,346.750
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,90.33,1100.00,1345.06,1254.73,-1791.765,3227.155
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,45.60,137.50,511.00,465.40,-652.500,1209.100
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,75.00,400.00,900.00,825.00,-1162.500,2137.500
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,45.60,137.50,511.00,465.40,-652.500,1209.100
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,27.75,55.00,160.00,132.25,-170.625,358.375


In [10]:
df_iqr['otlr_iqr'] = ((df_iqr['valorpago'] < df_iqr['limite_inferior']) | (df_iqr['valorpago'] > df_iqr['limite_superior'])).astype(int)

df_iqr

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,Q1,mediana,Q3,IQR,limite_inferior,limite_superior,otlr_iqr
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,49.25,150.00,210.00,160.75,-191.875,451.125,0
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,746.32,755.27,765.26,18.94,717.910,793.670,0
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,150.00,150.00,200.00,50.00,75.000,275.000,0
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,35.50,92.50,160.00,124.50,-151.250,346.750,0
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,90.33,1100.00,1345.06,1254.73,-1791.765,3227.155,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,45.60,137.50,511.00,465.40,-652.500,1209.100,0
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,75.00,400.00,900.00,825.00,-1162.500,2137.500,0
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,45.60,137.50,511.00,465.40,-652.500,1209.100,0
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,27.75,55.00,160.00,132.25,-170.625,358.375,0


In [11]:
# Group by and aggregate
grouped_iqr = df_iqr.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria']).agg(
    otlr_iqr=('otlr_iqr', 'sum'),
    qtd_lancamentos=('qtd_lancamentos', 'sum')
).reset_index()

# Calculate the percentage of outliers
grouped_iqr['pct_otlr_iqr'] = (grouped_iqr['otlr_iqr'] / grouped_iqr['qtd_lancamentos']).round(2)

grouped_iqr

,id,nome_unidade,categoria,subcategoria,otlr_iqr,qtd_lancamentos,pct_otlr_iqr
0,0.0,CENTRAL AMORSAÚDE,Funcionários,Vale Transporte,0,2,0.00
1,18263.0,AmorSaúde Carapicuíba,Administrativas,Advogados,0,1,0.00
2,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel Imóvel,0,100,0.00
3,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel de Equipamentos,0,90,0.00
4,18263.0,AmorSaúde Carapicuíba,Administrativas,Alvarás,0,26,0.00
...,...,...,...,...,...,...,...
33936,19990.0,AmorSaúde Piracicaba Vila Sônia,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,0,2,0.00
33937,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Consultas,7,65,0.11
33938,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Procedimentos,2,30,0.07
33939,19990.0,AmorSaúde Piracicaba Vila Sônia,Sócios,Transação Entre Contas Médica e Odonto,0,4,0.00


In [12]:
# obtendo o desvio padrão para cada linha de despesa de cada unidade
df_std = df.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['valorpago'].std().reset_index()
df_std = df_std.rename(columns={'valorpago': 'dvp'})
df_zscore = df.merge(df_std, on=['id', 'nome_unidade', 'categoria', 'subcategoria'], how='left')
df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493
...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625


In [13]:
# obtendo a média para cada linha de despesa de cada unidade
df_mean = df_zscore.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['valorpago'].mean().reset_index()
df_mean = df_mean.rename(columns={'valorpago': 'media'})
df_zscore = df_zscore.merge(df_mean, on=['id', 'nome_unidade', 'categoria', 'subcategoria'], how='left')
df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662
...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565


In [14]:
# obtendo a mediana para cada despesa de cada unidade
df_median = df_zscore.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['valorpago'].median().reset_index()
df_median.rename(columns={'valorpago': 'mediana'}, inplace=True)

df_zscore = pd.merge(df_zscore, df_median, how='left', on=['id', 'nome_unidade', 'categoria', 'subcategoria'])

In [15]:
#criando coluna de diferença absoluta
df_zscore['diff_abs'] = (df_zscore['valorpago'] - df_zscore['mediana']).abs()
df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465,150.00,70.00
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889,755.27,5.29
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167,150.00,50.00
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522,92.50,34.50
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662,1100.00,13.25
...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000,137.50,112.50
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128,400.00,375.00
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000,137.50,663.50
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565,55.00,5.00


In [16]:
df_mad = df_zscore.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria'])['diff_abs'].median().reset_index()
df_mad.rename(columns={'diff_abs': 'mda'}, inplace=True)
df_zscore = df_zscore.merge(df_mad, on=['id', 'nome_unidade', 'categoria', 'subcategoria'], how='left')
df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs,mda
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465,150.00,70.00,70.00
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889,755.27,5.29,8.95
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167,150.00,50.00,50.00
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522,92.50,34.50,57.50
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662,1100.00,13.25,400.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000,137.50,112.50,137.50
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128,400.00,375.00,350.00
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000,137.50,663.50,137.50
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565,55.00,5.00,36.50


In [17]:
#calculando o zscore modificado
calculate_modified_zscore = lambda x, median, mad: ((x - median) / mad) * 0.6745 if mad != 0 else None
df_zscore['mod_zscore'] = df_zscore.apply(lambda row: calculate_modified_zscore(row['valorpago'], row['mediana'], row['mda']), axis=1)

df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs,mda,mod_zscore
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465,150.00,70.00,70.00,0.674500
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889,755.27,5.29,8.95,-0.398671
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167,150.00,50.00,50.00,0.674500
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522,92.50,34.50,57.50,-0.404700
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662,1100.00,13.25,400.00,-0.022343
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000,137.50,112.50,137.50,0.551864
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128,400.00,375.00,350.00,-0.722679
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000,137.50,663.50,137.50,3.254769
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565,55.00,5.00,36.50,-0.092397


In [18]:
#obtendo coluna de zscore para casos em que mad é 0
calculate_zscore = lambda x, mean, std: ((x - mean) / std) if std != 0 else None
df_zscore['zscore'] = df_zscore.apply(lambda row: calculate_zscore(row['valorpago'], row['media'], row['dvp']), axis=1)

df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs,mda,mod_zscore,zscore
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465,150.00,70.00,70.00,0.674500,0.985540
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889,755.27,5.29,8.95,-0.398671,-0.677713
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167,150.00,50.00,50.00,0.674500,0.881181
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522,92.50,34.50,57.50,-0.404700,-0.496530
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662,1100.00,13.25,400.00,-0.022343,0.341138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000,137.50,112.50,137.50,0.551864,-0.273186
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128,400.00,375.00,350.00,-0.722679,-1.070809
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000,137.50,663.50,137.50,3.254769,0.910424
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565,55.00,5.00,36.50,-0.092397,-0.440699


In [19]:
# criando flag de outlier do zscore modificado
df_zscore['otlr_mod_zscore'] = df_zscore['mod_zscore'].apply(lambda x: 1 if abs(x) > 3.5 else 0)
df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs,mda,mod_zscore,zscore,otlr_mod_zscore
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465,150.00,70.00,70.00,0.674500,0.985540,0
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889,755.27,5.29,8.95,-0.398671,-0.677713,0
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167,150.00,50.00,50.00,0.674500,0.881181,0
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522,92.50,34.50,57.50,-0.404700,-0.496530,0
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662,1100.00,13.25,400.00,-0.022343,0.341138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000,137.50,112.50,137.50,0.551864,-0.273186,0
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128,400.00,375.00,350.00,-0.722679,-1.070809,0
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000,137.50,663.50,137.50,3.254769,0.910424,0
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565,55.00,5.00,36.50,-0.092397,-0.440699,0


In [20]:
# criando flag de outlier do zscore
df_zscore['otlr_zscore'] = df_zscore['zscore'].apply(lambda x: 1 if abs(x) > 3. else 0)
df_zscore

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs,mda,mod_zscore,zscore,otlr_mod_zscore,otlr_zscore
0,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Vale Alimentação/Refeição,220.00,1,86.774278,134.480465,150.00,70.00,70.00,0.674500,0.985540,0,0
1,2021-01-01,19773.0,AmorSaúde RJ Santa Cruz,Administrativas,Seguro Predial,749.98,1,9.087748,756.138889,755.27,5.29,8.95,-0.398671,-0.677713,0,0
2,2021-01-01,19626.0,AmorSaúde Porto Velho,Administrativas,Transportadora,200.00,1,56.374134,150.324167,150.00,50.00,50.00,0.674500,0.881181,0,0
3,2021-01-01,19566.0,AmorSaúde Araçatuba,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,58.00,1,152.374588,133.658522,92.50,34.50,57.50,-0.404700,-0.496530,0,0
4,2021-01-01,19626.0,AmorSaúde Porto Velho,Funcionários,Folha Mensal - Salário,1086.75,1,670.518493,858.010662,1100.00,13.25,400.00,-0.022343,0.341138,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491766,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,250.00,1,465.525016,377.175000,137.50,112.50,137.50,0.551864,-0.273186,0,0
2491767,2023-08-23,19608.0,AmorSaúde Itanhaém,Repasses,Repasse de Consultas,25.00,1,451.004135,507.939128,400.00,375.00,350.00,-0.722679,-1.070809,0,0
2491768,2023-08-23,19821.0,AmorSaúde SP Grajaú,Repasses,Repasse de Consultas,801.00,1,465.525016,377.175000,137.50,663.50,137.50,3.254769,0.910424,0,0
2491769,2023-08-23,19640.0,AmorSaúde Belém Bengui,Repasses,Repasse de Consultas,50.00,1,167.607625,123.864565,55.00,5.00,36.50,-0.092397,-0.440699,0,0


In [21]:
grouped_zscore = df_zscore.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria']).agg({
    'otlr_mod_zscore': 'sum',
    'otlr_zscore': 'sum',
    'qtd_lancamentos': 'sum'
}).reset_index()
#df_grouped = df_merge.groupby(['id', 'nome_unidade', 'categoria', 'subcategoria']).sum().reset_index()

grouped_zscore

,id,nome_unidade,categoria,subcategoria,otlr_mod_zscore,otlr_zscore,qtd_lancamentos
0,0.0,CENTRAL AMORSAÚDE,Funcionários,Vale Transporte,0,0,2
1,18263.0,AmorSaúde Carapicuíba,Administrativas,Advogados,0,0,1
2,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel Imóvel,32,2,100
3,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel de Equipamentos,0,0,90
4,18263.0,AmorSaúde Carapicuíba,Administrativas,Alvarás,0,0,26
...,...,...,...,...,...,...,...
33936,19990.0,AmorSaúde Piracicaba Vila Sônia,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,0,0,2
33937,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Consultas,0,0,65
33938,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Procedimentos,1,1,30
33939,19990.0,AmorSaúde Piracicaba Vila Sônia,Sócios,Transação Entre Contas Médica e Odonto,0,0,4


In [22]:
grouped_zscore['pct_otlr_mod_zscore'] = (grouped_zscore['otlr_mod_zscore'] / grouped_zscore['qtd_lancamentos']).round(2)
grouped_zscore['pct_otlr_zscore'] = (grouped_zscore['otlr_zscore'] / grouped_zscore['qtd_lancamentos']).round(2)
grouped_zscore

,id,nome_unidade,categoria,subcategoria,otlr_mod_zscore,otlr_zscore,qtd_lancamentos,pct_otlr_mod_zscore,pct_otlr_zscore
0,0.0,CENTRAL AMORSAÚDE,Funcionários,Vale Transporte,0,0,2,0.00,0.00
1,18263.0,AmorSaúde Carapicuíba,Administrativas,Advogados,0,0,1,0.00,0.00
2,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel Imóvel,32,2,100,0.32,0.02
3,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel de Equipamentos,0,0,90,0.00,0.00
4,18263.0,AmorSaúde Carapicuíba,Administrativas,Alvarás,0,0,26,0.00,0.00
...,...,...,...,...,...,...,...,...,...
33936,19990.0,AmorSaúde Piracicaba Vila Sônia,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,0,0,2,0.00,0.00
33937,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Consultas,0,0,65,0.00,0.00
33938,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Procedimentos,1,1,30,0.03,0.03
33939,19990.0,AmorSaúde Piracicaba Vila Sônia,Sócios,Transação Entre Contas Médica e Odonto,0,0,4,0.00,0.00


In [23]:
grouped_zscore = grouped_zscore.drop(labels='qtd_lancamentos', axis=1)
grouped_otlr = pd.merge(grouped_iqr, grouped_zscore, on=['id', 'nome_unidade', 'categoria', 'subcategoria'], how='left')
grouped_otlr

,id,nome_unidade,categoria,subcategoria,otlr_iqr,qtd_lancamentos,pct_otlr_iqr,otlr_mod_zscore,otlr_zscore,pct_otlr_mod_zscore,pct_otlr_zscore
0,0.0,CENTRAL AMORSAÚDE,Funcionários,Vale Transporte,0,2,0.00,0,0,0.00,0.00
1,18263.0,AmorSaúde Carapicuíba,Administrativas,Advogados,0,1,0.00,0,0,0.00,0.00
2,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel Imóvel,0,100,0.00,32,2,0.32,0.02
3,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel de Equipamentos,0,90,0.00,0,0,0.00,0.00
4,18263.0,AmorSaúde Carapicuíba,Administrativas,Alvarás,0,26,0.00,0,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
33936,19990.0,AmorSaúde Piracicaba Vila Sônia,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,0,2,0.00,0,0,0.00,0.00
33937,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Consultas,7,65,0.11,0,0,0.00,0.00
33938,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Procedimentos,2,30,0.07,1,1,0.03,0.03
33939,19990.0,AmorSaúde Piracicaba Vila Sônia,Sócios,Transação Entre Contas Médica e Odonto,0,4,0.00,0,0,0.00,0.00


In [24]:
# Define the desired column order
desired_order = ['id', 'nome_unidade', 'categoria', 'subcategoria', 'qtd_lancamentos', 'otlr_iqr', 'pct_otlr_iqr', 'otlr_zscore', 'pct_otlr_zscore', 'otlr_mod_zscore', 'pct_otlr_mod_zscore']

# Reorder the columns in the DataFrame
grouped_otlr = grouped_otlr[desired_order]

grouped_otlr

,id,nome_unidade,categoria,subcategoria,qtd_lancamentos,otlr_iqr,pct_otlr_iqr,otlr_zscore,pct_otlr_zscore,otlr_mod_zscore,pct_otlr_mod_zscore
0,0.0,CENTRAL AMORSAÚDE,Funcionários,Vale Transporte,2,0,0.00,0,0.00,0,0.00
1,18263.0,AmorSaúde Carapicuíba,Administrativas,Advogados,1,0,0.00,0,0.00,0,0.00
2,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel Imóvel,100,0,0.00,2,0.02,32,0.32
3,18263.0,AmorSaúde Carapicuíba,Administrativas,Aluguel de Equipamentos,90,0,0.00,0,0.00,0,0.00
4,18263.0,AmorSaúde Carapicuíba,Administrativas,Alvarás,26,0,0.00,0,0.00,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
33936,19990.0,AmorSaúde Piracicaba Vila Sônia,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,2,0,0.00,0,0.00,0,0.00
33937,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Consultas,65,7,0.11,0,0.00,0,0.00
33938,19990.0,AmorSaúde Piracicaba Vila Sônia,Repasses,Repasse de Procedimentos,30,2,0.07,1,0.03,1,0.03
33939,19990.0,AmorSaúde Piracicaba Vila Sônia,Sócios,Transação Entre Contas Médica e Odonto,4,0,0.00,0,0.00,0,0.00


In [25]:
df_iqr = df_iqr.drop(labels='mediana', axis=1)

#df_validacao_btts.to_excel('despesas_batatais.xlsx', index=False)

In [26]:
df_iqr_btts = df_iqr.loc[df_iqr['id']==19957]
df_zscore_btts = df_zscore.loc[df_zscore['id']==19957]


In [27]:
df_iqr_btts = df_iqr_btts.sort_values(by=['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos'])
df_zscore_btts = df_zscore_btts.sort_values(by=['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos'])

columns_to_drop = ['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos']  # List the columns you want to drop
df_zscore_btts = df_zscore_btts.drop(columns=columns_to_drop)

# Merge the dataframes again
#df_validacao_btts = df_iqr_btts.merge(df_zscore_btts, on=['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos'], how='left')
#df_validacao_btts = df_iqr_btts.merge(df_zscore_btts, left_index=True, right_index=True, how='left')
df_validacao_btts = df_iqr_btts.join(df_zscore_btts, how='left')
df_validacao_btts

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,Q1,Q3,IQR,...,otlr_iqr,dvp,media,mediana,diff_abs,mda,mod_zscore,zscore,otlr_mod_zscore,otlr_zscore
1526805,2022-12-01,19957.0,AmorSaúde Batatais,Administrativas,Manutenção Predial,291.00,1,169.5000,728.00,558.5000,...,0,587.454112,501.333333,291.00,0.00,243.00,0.000000,-0.358042,0,0
1525722,2022-12-01,19957.0,AmorSaúde Batatais,Financeiras,Pagamento de Empréstimos Bancários,3225.00,1,3225.0000,3225.00,0.0000,...,0,NaN,3225.000000,3225.00,0.00,0.00,NaN,NaN,0,0
1523730,2022-12-01,19957.0,AmorSaúde Batatais,Funcionários,Vale Alimentação/Refeição,519.20,1,519.2000,613.60,94.4000,...,0,785.947055,649.585075,542.80,23.60,47.20,-0.337250,-0.165895,0,0
1523732,2022-12-01,19957.0,AmorSaúde Batatais,Funcionários,Vale Alimentação/Refeição,519.20,1,519.2000,613.60,94.4000,...,0,785.947055,649.585075,542.80,23.60,47.20,-0.337250,-0.165895,0,0
1523798,2022-12-01,19957.0,AmorSaúde Batatais,Funcionários,Vale Alimentação/Refeição,519.20,1,519.2000,613.60,94.4000,...,0,785.947055,649.585075,542.80,23.60,47.20,-0.337250,-0.165895,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490514,2023-08-22,19957.0,AmorSaúde Batatais,Administrativas,Materiais de Limpeza,25.40,1,77.0475,480.66,403.6125,...,0,1188.660696,729.566667,134.18,108.78,92.81,-0.790563,-0.592403,0,0
2491320,2023-08-22,19957.0,AmorSaúde Batatais,Administrativas,Materiais de Limpeza,304.47,1,77.0475,480.66,403.6125,...,0,1188.660696,729.566667,134.18,170.29,92.81,1.237589,-0.357627,0,0
2489392,2023-08-22,19957.0,AmorSaúde Batatais,Repasses,Repasse de Consultas,86.28,1,32.0000,181.50,149.5000,...,0,195.181883,152.381538,72.00,14.28,42.00,0.229330,-0.338666,0,0
2489390,2023-08-22,19957.0,AmorSaúde Batatais,Royalties,Royalties,15.48,1,18.2800,81.66,63.3800,...,0,148.973214,68.751156,46.24,30.76,29.98,-0.692049,-0.357589,0,0


In [29]:
df_iqr_bhb = df_iqr.loc[df_iqr['id']==19543]
df_iqr_bhb

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,Q1,Q3,IQR,limite_inferior,limite_superior,otlr_iqr
367,2021-01-04,19543.0,AmorSaúde BH Barreiro,Financeiras,Manutenção Contas PIX/TED/DOCS/IOF e outros,13.84,1,106.0800,483.5475,377.4675,-460.12125,1049.74875,0
673,2021-01-04,19543.0,AmorSaúde BH Barreiro,Financeiras,Custo De Cartão De Crédito/Débito (Juros/Taxa/...,62.73,1,66.8350,395.1600,328.3250,-425.65250,887.64750,0
1825,2021-01-05,19543.0,AmorSaúde BH Barreiro,Administrativas,Transportadora,351.00,1,14.2125,69.3300,55.1175,-68.46375,152.00625,1
2071,2021-01-05,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,8.00,1,29.8500,156.0000,126.1500,-159.37500,345.22500,0
2075,2021-01-05,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,32.00,1,29.8500,156.0000,126.1500,-159.37500,345.22500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488837,2023-08-22,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Consultas Médicas,35.00,1,28.0000,35.2500,7.2500,17.12500,46.12500,0
2490716,2023-08-22,19543.0,AmorSaúde BH Barreiro,Administrativas,Brindes e Donativos,77.00,1,30.0000,35.9000,5.9000,21.15000,44.75000,1
2490718,2023-08-22,19543.0,AmorSaúde BH Barreiro,Administrativas,Materiais de Escritório,47.55,1,22.1250,179.9950,157.8700,-214.68000,416.80000,0
2490721,2023-08-22,19543.0,AmorSaúde BH Barreiro,Repasses,Repasse de Consultas,20.00,1,15.0000,37.0000,22.0000,-18.00000,70.00000,0


In [30]:
df_zscore_bhb = df_zscore.loc[df_zscore['id']==19543]
df_zscore_bhb

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,dvp,media,mediana,diff_abs,mda,mod_zscore,zscore,otlr_mod_zscore,otlr_zscore
367,2021-01-04,19543.0,AmorSaúde BH Barreiro,Financeiras,Manutenção Contas PIX/TED/DOCS/IOF e outros,13.84,1,2522.271111,852.062895,352.56,338.72,246.480,-0.926918,-0.332329,0,0
673,2021-01-04,19543.0,AmorSaúde BH Barreiro,Financeiras,Custo De Cartão De Crédito/Débito (Juros/Taxa/...,62.73,1,656.461995,432.988182,395.16,332.43,262.250,-0.855001,-0.564021,0,0
1825,2021-01-05,19543.0,AmorSaúde BH Barreiro,Administrativas,Transportadora,351.00,1,314.918650,130.052576,30.86,320.14,20.895,10.334263,0.701602,1,0
2071,2021-01-05,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,8.00,1,147.740886,118.496390,68.00,60.00,49.300,-0.820892,-0.747907,0,0
2075,2021-01-05,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Exames e Procedimentos Médicos,32.00,1,147.740886,118.496390,68.00,36.00,49.300,-0.492535,-0.585460,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488837,2023-08-22,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Consultas Médicas,35.00,1,80.353862,51.531585,32.00,3.00,4.000,0.505875,-0.205735,0,0
2490716,2023-08-22,19543.0,AmorSaúde BH Barreiro,Administrativas,Brindes e Donativos,77.00,1,346.732818,77.765023,35.00,42.00,5.000,5.665800,-0.002206,1,0
2490718,2023-08-22,19543.0,AmorSaúde BH Barreiro,Administrativas,Materiais de Escritório,47.55,1,1192.197083,252.999600,52.75,5.20,40.100,-0.087466,-0.172329,0,0
2490721,2023-08-22,19543.0,AmorSaúde BH Barreiro,Repasses,Repasse de Consultas,20.00,1,729.827392,194.336987,26.00,6.00,11.000,-0.367909,-0.238874,0,0


In [31]:
df_iqr_bhb = df_iqr_bhb.sort_values(by=['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos'])
df_zscore_bhb = df_zscore_bhb.sort_values(by=['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos'])

columns_to_drop = ['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos']  # List the columns you want to drop
df_zscore_bhb = df_zscore_bhb.drop(columns=columns_to_drop)

# Merge the dataframes again
#df_validacao_btts = df_iqr_btts.merge(df_zscore_btts, on=['datapagamento', 'id', 'nome_unidade', 'categoria', 'subcategoria', 'valorpago', 'qtd_lancamentos'], how='left')
#df_validacao_btts = df_iqr_btts.merge(df_zscore_btts, left_index=True, right_index=True, how='left')
df_validacao_bhb = df_iqr_bhb.join(df_zscore_bhb, how='left')
df_validacao_bhb

,datapagamento,id,nome_unidade,categoria,subcategoria,valorpago,qtd_lancamentos,Q1,Q3,IQR,...,otlr_iqr,dvp,media,mediana,diff_abs,mda,mod_zscore,zscore,otlr_mod_zscore,otlr_zscore
673,2021-01-04,19543.0,AmorSaúde BH Barreiro,Financeiras,Custo De Cartão De Crédito/Débito (Juros/Taxa/...,62.73,1,66.8350,395.1600,328.3250,...,0,656.461995,432.988182,395.16,332.43,262.250,-0.855001,-0.564021,0,0
367,2021-01-04,19543.0,AmorSaúde BH Barreiro,Financeiras,Manutenção Contas PIX/TED/DOCS/IOF e outros,13.84,1,106.0800,483.5475,377.4675,...,0,2522.271111,852.062895,352.56,338.72,246.480,-0.926918,-0.332329,0,0
2784,2021-01-05,19543.0,AmorSaúde BH Barreiro,Administrativas,Publicidade e Propaganda,800.00,1,370.0000,800.0000,430.0000,...,0,442.579763,650.707143,745.00,55.00,360.050,0.103034,0.337324,0,0
1825,2021-01-05,19543.0,AmorSaúde BH Barreiro,Administrativas,Transportadora,351.00,1,14.2125,69.3300,55.1175,...,1,314.918650,130.052576,30.86,320.14,20.895,10.334263,0.701602,1,0
2497,2021-01-05,19543.0,AmorSaúde BH Barreiro,Funcionários,Folha Mensal - Salário,24.00,1,156.1000,2064.8075,1908.7075,...,0,3395.155019,2082.852744,1489.00,1465.00,1043.000,-0.947404,-0.606409,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490718,2023-08-22,19543.0,AmorSaúde BH Barreiro,Administrativas,Materiais de Escritório,47.55,1,22.1250,179.9950,157.8700,...,0,1192.197083,252.999600,52.75,5.20,40.100,-0.087466,-0.172329,0,0
2488837,2023-08-22,19543.0,AmorSaúde BH Barreiro,Outras Despesas,Ressarcimento de Consultas Médicas,35.00,1,28.0000,35.2500,7.2500,...,0,80.353862,51.531585,32.00,3.00,4.000,0.505875,-0.205735,0,0
2490721,2023-08-22,19543.0,AmorSaúde BH Barreiro,Repasses,Repasse de Consultas,20.00,1,15.0000,37.0000,22.0000,...,0,729.827392,194.336987,26.00,6.00,11.000,-0.367909,-0.238874,0,0
2488011,2023-08-22,19543.0,AmorSaúde BH Barreiro,Repasses,Repasse de Consultas,46.00,1,15.0000,37.0000,22.0000,...,0,729.827392,194.336987,26.00,20.00,11.000,1.226364,-0.203249,0,0


In [32]:
df_validacao_bhb.to_excel('despesas_bhb.xlsx', index=False)
df_validacao_btts.to_excel('despesas_btts.xlsx', index=False)

In [33]:
grouped_otlr.to_excel('despesas_agrupadas.xlsx', index=False)


: 